# Imports

In [1]:
import requests as re
import json
import pandas as pd
import sqlalchemy
import pymysql
from sqlalchemy import create_engine


In [2]:
from datetime import datetime

# Connection to Flights API

In [3]:
response = re.get("http://aviation-edge.com/v2/public/flights?key=8bbf2c-a12972&depIata=BCN")
response.headers

{'Date': 'Thu, 21 Mar 2019 14:56:42 GMT', 'Server': 'Apache', 'Access-Control-Allow-Origin': '*', 'Access-Control-Allow-Headers': 'X-Requested-With, Content-Type, Accept, Origin, Authorization', 'Access-Control-Allow-Methods': 'GET, POST, PUT, DELETE, OPTIONS', 'Cache-Control': 'max-age=172800', 'Expires': 'Sat, 23 Mar 2019 14:56:42 GMT', 'Vary': 'Accept-Encoding,User-Agent', 'Content-Encoding': 'gzip', 'Content-Length': '6242', 'Keep-Alive': 'timeout=5, max=100', 'Connection': 'Keep-Alive', 'Content-Type': 'text/html; charset=UTF-8'}

In [4]:
flights_api = response.json()
print(json.dumps(flights_api,sort_keys=True, indent=4))

[
    {
        "aircraft": {
            "iataCode": "A20N",
            "icao24": "",
            "icaoCode": "A20N",
            "regNumber": "ECNAX"
        },
        "airline": {
            "iataCode": "VY",
            "icaoCode": "VLG"
        },
        "arrival": {
            "iataCode": "AMS",
            "icaoCode": "EHAM"
        },
        "departure": {
            "iataCode": "BCN",
            "icaoCode": "LEBL"
        },
        "flight": {
            "iataNumber": "VY83QE",
            "icaoNumber": "VLG83QE",
            "number": "83QE"
        },
        "geography": {
            "altitude": 2721.04,
            "direction": 10.2,
            "latitude": 41.3926,
            "longitude": 2.27602
        },
        "speed": {
            "horizontal": 502.732,
            "isGround": 0,
            "vertical": 0
        },
        "status": "en-route",
        "system": {
            "squawk": "0",
            "updated": "1553180015"
        }
    },
    {
   

# Generate Series with the information we need

In [5]:
plane_iata = pd.Series([plane['aircraft']['iataCode'] for plane in flights_api])
plane_icao = pd.Series([plane['aircraft']['icaoCode'] for plane in flights_api])
reg_num = pd.Series([plane['aircraft']['regNumber'] for plane in flights_api])
airline_iata = pd.Series([plane['airline']['iataCode'] for plane in flights_api])
airline_icao = pd.Series([plane['airline']['icaoCode'] for plane in flights_api])
dep_iata = pd.Series([plane['departure']['iataCode'] for plane in flights_api])
arr_iata = pd.Series([plane['arrival']['iataCode'] for plane in flights_api])
flight = pd.Series([plane['flight']['number'] for plane in flights_api])
latitude = pd.Series([plane['geography']['latitude'] for plane in flights_api])
longitude = pd.Series([plane['geography']['longitude'] for plane in flights_api])
hor_speed = pd.Series([plane['speed']['horizontal'] for plane in flights_api])
ver_speed = pd.Series([plane['speed']['vertical'] for plane in flights_api])
status = pd.Series([plane['status'] for plane in flights_api])
updated_time = pd.Series([plane['system']['updated'] for plane in flights_api])

# Create DataFrame from the Series

In [6]:
flights = pd.DataFrame({'aircraft_iata': plane_iata, 'aircraft_icao': plane_icao, 'reg_number': reg_num,'airline_iata': airline_iata, 'airline_icao': airline_icao, 'departure': dep_iata, 'arrival': arr_iata, 'flight_number': flight, 'latitude': latitude, 'longitude': longitude, 'horizontal_speed': hor_speed, 'vertical_speed': ver_speed, 'status': status, 'updated_time': updated_time})

In [7]:
flights

,aircraft_iata,aircraft_icao,reg_number,airline_iata,airline_icao,departure,arrival,flight_number,latitude,longitude,horizontal_speed,vertical_speed,status,updated_time
0,A20N,A20N,ECNAX,VY,VLG,BCN,AMS,83QE,41.3926,2.276020,502.732,0.00000,en-route,1553180015
1,B738,B738,PHHZV,HV,TRA,BCN,RTM,27J,51.5666,4.832800,544.194,14.63040,en-route,1553180015
2,A319,A319,GEZFV,U2,EZY,BCN,NCL,6402,53.4764,-1.412530,759.320,0.00000,en-route,1553179976
3,A321,A321,ECMGY,VY,VLG,BCN,CPH,1872,53.7537,10.816500,844.512,0.00000,en-route,1553179973
4,A359,A359,A7AMA,QR,QTR,BCN,DOH,146,41.1102,3.809940,759.320,0.00000,en-route,1553179965
5,A321,A321,HBIOH,LX,SWR,BCN,ZRH,1955,43.1275,5.701510,783.396,0.00000,en-route,1553179942
6,DH8D,DH8D,LXLQB,LG,LGL,BCN,LUX,3592,43.6982,5.875510,613.012,0.00000,en-route,1553179942
7,A320,A320,ECJGM,VY,VLG,BCN,MRS,1508,41.2850,2.074170,0.000,0.00000,landed,1553179938
8,B738,B738,LNNGG,DY,NAX,BCN,OSL,41B,57.4461,7.923700,879.408,0.00000,en-route,1553179870
9,B738,B738,EIDAJ,FR,RYR,BCN,BHX,76NA,52.2729,-1.064800,569.268,-18.72000,en-route,1553179870


# Converting timestamps into datetimes

In [8]:
flights['updated_time'] = flights['updated_time'].apply(lambda x: datetime.utcfromtimestamp(int(x)).strftime('%Y-%m-%d %H:%M:%S'))

# Convert speeds values into floats, and adding total speed (pitagoras)

In [9]:
flights['horizontal_speed'] = flights['horizontal_speed'].apply(float)
flights['vertical_speed'] = flights['vertical_speed'].apply(float)
flights['total_speed'] = round((flights['horizontal_speed']**2 + flights['vertical_speed']**2)**0.5,2)

# Checkout there are not null values

In [10]:
flights.isnull().sum()

aircraft_iata       0
aircraft_icao       0
reg_number          0
airline_iata        0
airline_icao        0
departure           0
arrival             0
flight_number       0
latitude            0
longitude           0
horizontal_speed    0
vertical_speed      0
status              0
updated_time        0
total_speed         0
dtype: int64

# Export to Database

In [11]:
driver = 'mysql+pymysql'
ip = '35.246.218.28'
username = 'root'
password = 'root'
db = 'flight_radar'

cs  = f'{driver}://{username}:{password}@{ip}/{db}'
engine = create_engine(cs)

In [12]:
flights.to_sql(con=engine, name='flights_dx', if_exists='append')